In [2]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
from config import skey
import json

In [18]:
pd.options.display.float_format = '{:.2f}'.format


In [19]:
user_data = pd.read_csv('data/100k_ids.csv')
user_data.head()

,steamid,timecreated,gameserverip,cityid,loccityid,loccountrycode,locstatecode
0,nan,NaN,NaN,nan,nan,NaN,NaN
1,76561200000000000.00,9/10/2003 20:58,NaN,nan,43754.00,SE,26
2,nan,NaN,NaN,nan,nan,NaN,NaN
3,76561200000000000.00,9/10/2003 21:53,NaN,nan,nan,US,NaN
4,nan,NaN,NaN,nan,nan,NaN,NaN


In [14]:
user_ids = user_data.dropna(how='all')
user_ids.reset_index(drop=True, inplace=True)
user_ids = user_ids.drop(columns=['timecreated', 'gameserverip', 'cityid', 'loccityid'])
user_ids = user_ids.rename(columns={'steamid': 'Steam ID',
                                    'loccountrycode': 'Country',
                                    'locstatecode': 'State'})
user_ids.head()

,Steam ID,Country,State
0,7.656120e+16,SE,26
1,7.656120e+16,US,NaN
2,7.656120e+16,CA,BC
3,7.656120e+16,AU,VIC
4,7.656120e+16,US,WA


In [11]:
steam_ids = user_ids['Steam ID'].tolist()
print(len(steam_ids))

99999


In [17]:
#user_ids['Steam ID'] = user_ids['Steam ID'].apply(lambda x: '{:.2f}'.format(x))
user_ids.head()

,Steam ID,Country,State
0,76561200000000000.00,SE,26
1,76561200000000000.00,US,NaN
2,76561200000000000.00,CA,BC
3,76561200000000000.00,AU,VIC
4,76561200000000000.00,US,WA


In [17]:
player_summary_url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={skey}&steamids=76561198068112781"
response = requests.get(player_summary_url).json()
print(steam_ids[0])
print(response)

7.65612e+16
{'response': {'players': [{'steamid': '76561198068112781', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'All Up In It', 'profileurl': 'https://steamcommunity.com/profiles/76561198068112781/', 'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444.jpg', 'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444_medium.jpg', 'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444_full.jpg', 'avatarhash': '3f38bdc0c43491dcc3e7a7a19ab6e10254547444', 'lastlogoff': 1615412403, 'personastate': 1, 'primaryclanid': '103582791433454257', 'timecreated': 1343378535, 'personastateflags': 0, 'loccountrycode': 'US', 'locstatecode': 'CO'}]}}


In [21]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "response": {
        "players": [
            {
                "avatar": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444.jpg",
                "avatarfull": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444_full.jpg",
                "avatarhash": "3f38bdc0c43491dcc3e7a7a19ab6e10254547444",
                "avatarmedium": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/3f/3f38bdc0c43491dcc3e7a7a19ab6e10254547444_medium.jpg",
                "communityvisibilitystate": 3,
                "lastlogoff": 1615412403,
                "loccountrycode": "US",
                "locstatecode": "CO",
                "personaname": "All Up In It",
                "personastate": 1,
                "personastateflags": 0,
                "primaryclanid": "103582791433454257",
                "profilestate": 1,
                "profileurl": "h

In [22]:
owned_games_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid=76561198068112781&format=json"
game_list = requests.get(owned_games_url).json()
print(json.dumps(game_list, indent=2))

{
  "response": {
    "game_count": 30,
    "games": [
      {
        "appid": 4000,
        "playtime_forever": 342,
        "playtime_windows_forever": 126,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 28050,
        "playtime_forever": 1702,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 205790,
        "playtime_forever": 0,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 6370,
        "playtime_forever": 38,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 238960,
        "playtime_forever": 564,
        "playtime_windows_forever": 564,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid"

In [5]:
game_news_url = f"http://api.steampowered.com/ISteamNews/GetNewsForApp/v0002/?appid=4000&count=3&maxlength=300&format=json"
game_news_test = requests.get(game_news_url).json()
base_appid_url = "https://steamdb.info/app/"
#print(json.dumps(game_news_test, indent=2))

In [7]:
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [9]:
browser.visit(base_appid_url + "28050")
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [11]:
game_title = soup.find('h1').get_text()
print(game_title)

Deus Ex: Human Revolution
